In [ ]:
!pip install deeplake
!pip install llama-index
!pip install llama-index-llms-openai
!pip install llama-index-vector-stores-deeplake
!pip install sentence-transformers
!pip install langchain-google-genai

In [22]:

import os
import requests
import re
import openai

from google.colab import userdata
from bs4 import BeautifulSoup

In [15]:
LLM_PATH = './contents/'

## 1.Source Retrieval

In [16]:
class SourceRetrieval:
  def __init__(self):
    openai.api_key = userdata.get('OPENAI_API_KEY')
    os.environ['USER_AGENT'] = 'RAGUserAgent'
    os.environ['ACTIVELOOP_TOKEN']= userdata.get('ACTIVELOOP_TOKEN')


    self.urls = [
        "https://github.com/VisDrone/VisDrone-Dataset",
        "https://paperswithcode.com/dataset/visdrone",
        "https://openaccess.thecvf.com/content_ECCVW_2018/papers/11133/Zhu_VisDrone-DET2018_The_Vision_Meets_Drone_Object_Detection_in_Image_Challenge_ECCVW_2018_paper.pdf",
        "https://github.com/VisDrone/VisDrone2018-MOT-toolkit",
        "https://en.wikipedia.org/wiki/Object_detection",
        "https://en.wikipedia.org/wiki/Computer_vision",
        "https://en.wikipedia.org/wiki/Convolutional_neural_network",
        "https://en.wikipedia.org/wiki/Unmanned_aerial_vehicle",
        "https://www.faa.gov/uas/",
        "https://www.tensorflow.org/",
        "https://pytorch.org/",
        "https://keras.io/",
        "https://arxiv.org/abs/1804.06985",
        "https://arxiv.org/abs/2202.11983",
        "https://motchallenge.net/",
        "http://www.cvlibs.net/datasets/kitti/",
        "https://www.dronedeploy.com/",
        "https://www.dji.com/",
        "https://arxiv.org/",
        "https://openaccess.thecvf.com/",
        "https://roboflow.com/",
        "https://www.kaggle.com/",
        "https://paperswithcode.com/",
        "https://github.com/"
    ]

  def get_contents(self):
    def clean_text(content):
      content = re.sub(r'\[\s*(\d+|edit)\s*\]','',content)
      content = re.sub(r'[^\w\s\.]','',content)
      return content

    def fetch(url):
      try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        content = soup.find('div',{'class': 'mw-parser-output'}) or soup.find('div', {'id': 'content'})
        if content is None:
            return None

        for section_title in ['References', 'Bibliography', 'External links', 'See also', 'Notes']:
            section = content.find('span',id=section_title)
            while section:
                for sib in section.parent.find_next_siblings():
                    sib.decompose()
                section.parent.decompose()
                section = content.find('span',id=section_title)

        text = content.get_text(separator=' ', strip=True)
        text = clean_text(text)
        return text
      except requests.exceptions.RequestException as e:
        print(f'error from {url}: {e}')
        return None

    for url in self.urls:
      article = url.split('/')[-1].replace('.html','')

      filename = os.path.join(LLM_PATH, f'{article}.txt')
      clean_article = fetch(url)
      if clean_article:
          with open(filename,'w',encoding='utf-8') as file:
              file.write(clean_article)
              print(f'\tContent was written to {filename}')

    print('Content writing done...')



In [20]:
sr = SourceRetrieval()
sr.get_contents()

	Content was written to ./contents/Object_detection.txt
	Content was written to ./contents/Computer_vision.txt
	Content was written to ./contents/Convolutional_neural_network.txt
	Content was written to ./contents/Unmanned_aerial_vehicle.txt
	Content was written to ./contents/1804.06985.txt
	Content was written to ./contents/2202.11983.txt
	Content was written to ./contents/.txt
	Content was written to ./contents/.txt
	Content was written to ./contents/.txt
Content writing done...


## 2. Vectorizing

In [23]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from llama_index.core import StorageContext,\
                             VectorStoreIndex,\
                             SimpleDirectoryReader,\
                             Document

from llama_index.vector_stores.deeplake import DeepLakeVectorStore

#from pydantic.v1 import BaseModel,Field
#from typing import ClassVar
import deeplake


In [25]:
class VectorStore:
  def __init__(self):
    self.documents = SimpleDirectoryReader(LLM_PATH).load_data()
    self.vector_path = 'hub://pythoninaction/drone_1000'
    self.dataset_path = 'hub://pythoninaction/drone_1000'
    self.gemini_embedding  = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

  def vectorize(self):
    vector_store = DeepLakeVectorStore(dataset_path=self.dataset_path, overwrite=True)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(self.documents, storage_context=storage_context)
    ds = deeplake.load(self.dataset_path)
    return ds,index

In [26]:
vs = VectorStore()
ds,index = vs.vectorize()

Your Deep Lake dataset has been successfully created!


Uploading data to deeplake dataset.


100%|██████████| 83/83 [00:01<00:00, 65.89it/s] 


Dataset(path='hub://pythoninaction/drone_1000', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (83, 1)      str     None   
 metadata     json      (83, 1)      str     None   
 embedding  embedding  (83, 1536)  float32   None   
    id        text      (83, 1)      str     None   


\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/pythoninaction/drone_1000



-

hub://pythoninaction/drone_1000 loaded successfully.



In [27]:
import json
import numpy as np
import pandas as pd

In [28]:
print(type(ds))

<class 'deeplake.core.dataset.deeplake_cloud_dataset.DeepLakeCloudDataset'>


In [31]:
data_llm = {}
for tensor_name in ds.tensors:
  tensor_data = ds[tensor_name].numpy()

  if tensor_data.ndim > 1:
    data_llm[tensor_name] = [np.array(e).flatten().tolist() for e in tensor_data]
  else:
    if tensor_name == 'text':
      data_llm[tensor_name] = [t.tobytes().decode('utf-8') if t else "" for t in tensor_data]
    else:
      data_llm[tensor_name] = tensor_data.tolist()

df_llm = pd.DataFrame(data_llm)
df_llm

,embedding,id,metadata,text
0,"[-0.0005138011183589697, 0.01005316898226738, ...",[f7e743a2-1d49-4a5e-81e0-2416509783d5],[{'file_path': '/content/contents/1804.06985.t...,[High Energy Physics Theory arXiv1804.06985 h...
1,"[-0.014190182089805603, 0.01753275841474533, 0...",[857d57fe-9cd3-42f3-9218-3d686526f7d0],[{'file_path': '/content/contents/2202.11983.t...,[Computer Science Computer Vision and Pattern...
2,"[-0.017965424805879593, -0.0010230127954855561...",[db15360d-75c5-4244-9eff-35381f59aa94],[{'file_path': '/content/contents/Computer_vis...,[Computerized information extraction from imag...
3,"[-0.014219841919839382, -0.003627510741353035,...",[3e7f5219-d1de-42a4-91f9-f958b88c24b8],[{'file_path': '/content/contents/Computer_vis...,[By the 1990s some of the previous research to...
4,"[-0.01666109450161457, 0.002121962606906891, 0...",[aad57743-3e6d-47f2-91f8-79802e2b0247],[{'file_path': '/content/contents/Computer_vis...,[Some strands of computer vision research are ...
...,...,...,...,...
78,"[-0.003586906474083662, 4.198138776700944e-05,...",[cb67b055-0c75-493b-8ddf-9ae6f4b17a05],[{'file_path': '/content/contents/Unmanned_aer...,[Biological Invasions . 26 2 357363. doi 10.1...
79,"[-0.018677927553653717, -0.010565834119915962,...",[9885cce8-ecad-4bc2-ad3b-c58c9cfd8b97],[{'file_path': '/content/contents/Unmanned_aer...,[Journal of Rock Mechanics and Geotechnical En...
80,"[-0.016946764662861824, -0.010993395932018757,...",[b75fda4f-3eec-4134-8d27-5b0a0baaf083],[{'file_path': '/content/contents/Unmanned_aer...,[CNN . Archived from the original on 8 Novembe...
81,"[-0.015471203252673149, -0.005759623367339373,...",[a5fda48e-0c5d-463b-96ff-48edd2a59785],[{'file_path': '/content/contents/Unmanned_aer...,[Formation Control for UAVs Using a Flux Guide...


In [32]:
df_llm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   embedding  83 non-null     object
 1   id         83 non-null     object
 2   metadata   83 non-null     object
 3   text       83 non-null     object
dtypes: object(4)
memory usage: 2.7+ KB


In [33]:
df_llm['text'] = df_llm['text'].astype(str)
df_llm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   embedding  83 non-null     object
 1   id         83 non-null     object
 2   metadata   83 non-null     object
 3   text       83 non-null     object
dtypes: object(4)
memory usage: 2.7+ KB


In [35]:
document_llms = [Document(text=row['text'], doc_id=str(row['id'])) for _, row in df_llm.iterrows()]
document_llms[0]

Document(id_="['f7e743a2-1d49-4a5e-81e0-2416509783d5']", embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="['High Energy Physics  Theory arXiv1804.06985 hepth Submitted on 19 Apr 2018 Title A Near Horizon Extreme Binary Black Hole Geometry Authors Jacob Ciafre  Maria J. Rodriguez View a PDF of the paper titled A Near Horizon Extreme Binary Black Hole Geometry by Jacob Ciafre and Maria J. Rodriguez View PDF Abstract A new solution of fourdimensional vacuum General Relativity is presented. It describes the near horizon region of the extreme maximally spinning binary black hole system with two identical extreme Kerr black holes held in equilibrium by a massless strut. This is the first example of a nonsupersymmetric asymptotically flat near horizon extreme binary black hole geometry of two uncharged black holes. The black holes are corotating and the solution is uniquely specified by the mass. The binary extreme system has

## 3.Query Engine

In [36]:
from llama_index.core import VectorStoreIndex

In [38]:
vector_store_engine_llm = VectorStoreIndex.from_documents(document_llms)
vector_query_engine_llm = vector_store_engine_llm.as_query_engine(similarity_top_k=10, temperatur=0.1, num_output=1024)

In [40]:

user_input="How do drones recognize car?"
import time
import textwrap
#start the timer
start_time = time.time()
llm_response = vector_query_engine_llm.query(user_input)
# Stop the timer
end_time = time.time()
# Calculate and print the execution time
elapsed_time = end_time - start_time
print(f"Query execution time: {elapsed_time:.4f} seconds")

print(textwrap.fill(str(llm_response), 100))

Query execution time: 1.9603 seconds
Drones can recognize cars through various methods such as computer vision technology, machine
learning algorithms, and sensors like cameras. These technologies enable drones to identify and
track cars based on visual patterns, shapes, and features.


## 4. Visualize

In [41]:
import deeplake

In [50]:
class VisualizeVector:
  def __init__(self):
    self.dataset_path = 'hub://activeloop/visdrone-det-train'
    self.ds = deeplake.load(self.dataset_path)

  def createStructure(self):
    df = pd.DataFrame(columns=['image','boxes','labels'])

    for i, sample in enumerate(self.ds):
       df.loc[i, 'image'] = sample.images.tobytes()
       boxes_list = sample.boxes.numpy(aslist=True)
       df.loc[i, 'boxes'] = [box.tolist() for box in boxes_list]

       label_data = sample.labels.data()
       df.loc[i, 'labels'] = label_data['text']

    return df,self.ds



In [51]:
vv = VisualizeVector()
df,ds = vv.createStructure()

/

Opening dataset in read-only mode as you don't have write permissions.


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/visdrone-det-train



-

hub://activeloop/visdrone-det-train loaded successfully.



In [47]:
df

,image,boxes,labels
0,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[1221.0, 84.0, 16.0, 33.0], [1235.0, 71.0, 18...","[pedestrian, pedestrian, tricycle, pedestrian,..."
1,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[351.0, 936.0, 305.0, 114.0], [0.0, 818.0, 22...","[car, car, car, car, car, car, car, car, car, ..."
2,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[699.0, 716.0, 26.0, 54.0], [600.0, 604.0, 22...","[truck, car, van, car, car, car, car, car, car..."
3,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[417.0, 77.0, 57.0, 54.0], [387.0, 109.0, 31....","[ignored regions, ignored regions, car, car, c..."
4,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[794.0, 617.0, 97.0, 115.0], [803.0, 539.0, 7...","[car, car, car, car, car, car, car, car, car, ..."
...,...,...,...
6466,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[683.0, 710.0, 7.0, 6.0], [681.0, 717.0, 5.0,...","[car, car, car, car, awning-tricycle, car, car..."
6467,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[1761.0, 676.0, 207.0, 155.0], [1539.0, 648.0...","[car, car, car, car, car, van, car, car, car, ..."
6468,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[0.0, 501.0, 105.0, 93.0], [412.0, 594.0, 172...","[car, van, pedestrian, pedestrian, pedestrian,..."
6469,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[[200.0, 604.0, 112.0, 64.0], [311.0, 560.0, 1...","[van, truck, van, car, car, van, others, car, ..."


In [52]:
labels_list = ds.labels.info['class_names']
labels_list

['ignored regions',
 'pedestrian',
 'people',
 'bicycle',
 'car',
 'van',
 'truck',
 'tricycle',
 'awning-tricycle',
 'bus',
 'motor',
 'others']

## Multi-model structure